In [268]:
# %pip install selenium
import pandas as pd, requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from time import sleep


In [576]:
driver = webdriver.Chrome(executable_path="./Chromedriver/macos/chromedriver") 

In [581]:
url = 'https://primenow.amazon.com/storefront?merchantId=A2H9WG57X0QEGO&ref_=pn_gw_fs_1_A2H9WG57X0QEGO'
driver.get(url)

try: 
    zip_code = driver.find_element_by_id('lsPostalCode')
    zip_code.send_keys('90048')
    submit_zipcode = driver.find_element_by_class_name('a-button-input')
    submit_zipcode.click()
except:
    pass

In [434]:
soup = BeautifulSoup(driver.page_source,'lxml')
categories = soup.find_all("div",{"class": "categoryTile__tile__30ivY"})
category_list=[]
for cat in categories[-12:]:
    category={}
    category['category'] = cat.get_text()
    category['link']=(cat.find({"a":"href"})['href'])
    category_list.append(category)

In [556]:
category_list

[{'category': ' Beef, Poultry, Pork, & More ',
  'link': '/browse?node=17732797011&merchantId=A29S0I53TYUGGN&ref_=pn_sf_ct_17732797011_A29S0I53TYUGGN&pf_rd_r=NA99WEVBTYM5G2F44WX2&pf_rd_p=d3b50bf9-56a8-42d3-bb03-6157d0ee9975&pf_rd_s=desktop-center-8&pf_rd_i=A29S0I53TYUGGN&pf_rd_t=101&pf_rd_m=A35T2T8BW467U4'},
 {'category': ' Produce & Organic Produce ',
  'link': '/browse?node=17576164011&merchantId=A29S0I53TYUGGN&ref_=pn_sf_ct_17576164011_A29S0I53TYUGGN&pf_rd_r=NA99WEVBTYM5G2F44WX2&pf_rd_p=7f39377d-6c40-45b1-a2bb-982f670adf22&pf_rd_s=desktop-center-9&pf_rd_i=A29S0I53TYUGGN&pf_rd_t=101&pf_rd_m=A35T2T8BW467U4'},
 {'category': ' Seafood ',
  'link': '/browse?node=17732795011&merchantId=A29S0I53TYUGGN&ref_=pn_sf_ct_17732795011_A29S0I53TYUGGN&pf_rd_r=NA99WEVBTYM5G2F44WX2&pf_rd_p=206c03c1-c6e5-4393-9b32-206454b4f16c&pf_rd_s=desktop-center-10&pf_rd_i=A29S0I53TYUGGN&pf_rd_t=101&pf_rd_m=A35T2T8BW467U4'},
 {'category': ' Prepared Foods ',
  'link': '/browse?node=19387925011&merchantId=A29S0I53TY

In [577]:
product_list= []

In [582]:
for category in category_list:
    driver.get('https://primenow.amazon.com'+category['link'])
    print(f"Scraping category: {category['category']}")
    try:
          number_of_pages = int(driver.find_elements_by_xpath('//span[@class = "buttons__paginate-link-text__30gGN"]')[-1].text)
    except:
          number_of_pages = 1
    print(f'{number_of_pages} pages are getting scraped')
    for count in range(number_of_pages):
        if number_of_pages != 1:
            if count==0:
                next_page = driver.find_elements_by_xpath('//li[@class = "buttons__pagination-item__1mFvI"]')[-1]
                next_page_url = next_page.find_element_by_css_selector('a').get_attribute('href')
            else:
                driver.get(next_page_url)
            
            
            
        sleep(1)
        soup = BeautifulSoup(driver.page_source,'lxml')
        products = soup.find_all('li',attrs={'class': lambda e: e.startswith('product_grid__item') if e else False})
#       products = soup.find_all("li",{"class":"product_grid__item__20t1D"})
        print(len(products))
        for product in products:
            product_info={}
            # Products that are in one page 

    #         new_line = product.text.replace('\n', ',')
    #         product_info_text = new_line.split(',')
            brand_name_weight = product.find("div",attrs={"class": lambda e: e.startswith("text_truncate__root") if e else False}).get_text().split(',')
            if len(brand_name_weight)==3:
                product_info['brand']=brand_name_weight[0]
                product_info['name']=brand_name_weight[1]
                product_info['weight']=brand_name_weight[-1]
                product_info['brand_name_weight']= brand_name_weight
                product_info['Category'] = category['category']
                
            elif len(brand_name_weight)==4:
                product_info['brand']=brand_name_weight[0]
                product_info['name']=brand_name_weight[1]
                product_info['weight']=brand_name_weight[-1]
                product_info['brand_name_weight']= brand_name_weight
                product_info['Category'] = category['category']
                
            elif len(brand_name_weight)==2:   
                product_info['brand']='nan'
                product_info['name']=brand_name_weight[0]
                product_info['weight']= brand_name_weight[1]
                product_info['brand_name_weight']= brand_name_weight
                product_info['Category'] = category['category']
                
            else:
                product_info['brand']='nan'
                product_info['name']=brand_name_weight[0]
                product_info['weight']='nan'
                product_info['brand_name_weight']= brand_name_weight
                product_info['Category'] = category['category']
            print(f"scraping { product_info['name']}")



            #Full price of the product
            full = product.find('div',attrs={"class":lambda e:e.startswith("asin_price__priceFull")if e else False}).get_text()
            product_info['full_price']= full
            print(full)

            # Sale price of the product 
            try:
                sale = product.find("div",attrs = {'class':lambda e:e.startswith("asin_price__priceDiscount")if e else False}).get_text()
            except:
                sale = "not sale"

            product_info['sale_price'] = sale
            product_info['store'] = 'WholeFoods'


            product_list.append(product_info)
            
        next_page_url = next_page_url.replace(f'page={count+1}',f'page={count+2}')
        next_page_url = next_page_url.replace(f'_pg_{count+1}', f'_pg_{count+2}')

Scraping category:  Beef, Poultry, Pork, & More 
11 pages are getting scraped
30
scraping  Smokehouse Bacon
$4.99
scraping  Natural Oven Roasted Turkey Breast
$5.99
scraping  Organic Tofu Extra Firm
$1.79
scraping  Natural Uncured Turkey Pepperoni
$6.49
scraping  Natural Uncured Genoa Salami
$4.99
scraping  Natural Oven Roasted Turkey Breast Family Size
$7.99
scraping Chicken Breast Boneless Skinless Air Chilled Organic Step 3
$7.99
scraping  Organic Oven Roasted Turkey Breast
$6.99
scraping  Meatless Protein Packed Strips
$4.79
scraping Beef Loin New York Strip Steak Boneless Step 1
$13.99
scraping Beef Ribeye Steak Boneless Step 1
$13.99
scraping Beef Loin New York Strip Steak Pasture Raised Boneless Step 4
$13.99
scraping  Natural Uncured Black Forest Ham Family Size
$7.99
scraping  Organic Cracked Black Pepper Turkey Breast
$5.49
scraping  Natural Uncured Black Forest Ham
$5.99
scraping  Organic Applewood Smoked Turkey Breast
$5.49
scraping  No Sugar Added Bacon
$6.99
scraping  Nat

30
scraping Fra Mani
$16.99
scraping Beef Burger Cowboy
$6.99
scraping Chicken Breast Tenderloin Air Chilled Organic Step 3
$8.99
scraping 365 Everyday Value
$4.49
scraping  Tofu Shirataki
$1.99
scraping  Poblano Black Bean Plant Burgers
$4.99
scraping Beef Loin Flank Steak Pasture Raised Step 4
$15.99
scraping  Salami Calabrese Sliced
$5.99
scraping  Prosciutto Ridgetop Sliced
$8.99
scraping 365 Everyday Value
$2.99
scraping  Tuna Fishless In Water
$4.99
scraping  Prosciutto de San Daniele
$3.99
scraping  Organic Black Rice Burger
$2.99
scraping  Pepperoni Sliced
$7.99
scraping Chicken Breast Boneless Skinless Fillet Organic Tray Pack Step 3
$7.99
scraping  Sausage Pork Chorizo
$6.49
scraping  Gimme Lean Sausage
$4.69
scraping  Uncured Italian Dry Salami
$4.99
scraping  Steakhouse Smoked Uncured Bacon
$6.99
scraping Hearst Ranch
$6.99
scraping Pork Shoulder Butt Roast Bonless Organic Step 1
$7.99
scraping Pork Sausage Link Breakfast Country Previously Frozen
$6.99
scraping Chicken Gro

30
scraping Iceberg Lettuce Organic
$2.49
scraping Apple Gala Conventional
$1.79
scraping Berry Blackberries Organic
$3.99
scraping Pepper Bell Orange Conventional
$2.99
scraping Honeydew Melon Conventional
$3.99
scraping Grape Red Seedless Conventional
$2.49
scraping White Onion Organic
$1.49
scraping Celery Bunch Conventional
$1.49
scraping Tomato Regular Organic
$2.49
scraping Cauliflower Conventional
$2.49
scraping Red Cabbage Organic
$1.49
scraping Eggplant Regular Conventional
$1.69
scraping Potato Red Organic
$1.49
scraping Pineapple
$2.99
scraping Eggplant Regular Organic
$2.49
scraping  Organic Rainbow Carrots
$2.99
scraping Lettuce Red Leaf Organic
$1.99
scraping Corn White Conventional
$0.79
scraping Blueberry Organic
$3.99
scraping Plum Red Organic
$3.99
scraping Squash Spaghetti Organic
$1.99
scraping Leeks Organic
$2.99
scraping Apple Granny Smith Organic Whole Trade Guarantee
$2.49
scraping Organic Cherry Tomato
$3.99
scraping Organic Lettuce Boston Butter
$1.99
scraping

6
scraping Pepper Sweet Mini Organic
$4.99
scraping Squash Yellow Organic Whole Trade Guarantee
$2.99
scraping Pepper Green Pasilla Conventional
$2.99
scraping Radish Watermelon Chinese Conventional
$3.99
scraping Onion Yellow Conventional
$2.99
scraping Apple Braeburn Whole Trade Guarantee Organic
$2.69
Scraping category:  Seafood 
7 pages are getting scraped
30
scraping Salmon Fillet Atlantic Farm Raised
$9.99
scraping  Sockeye Salmon Fillet
$10.99
scraping  Cooked White Shrimp Previously Frozen 31
$14.99
scraping Shrimp White Farm Raised Peeled Previously Frozen 26 30
$9.99
scraping  New York-Style Nova Lox
$8.99
scraping  Atlantic Salmon Fillets
$19.99
scraping Tilapia Fillet Farm Raised
$7.99
scraping Fishpeople Wild Alaskan Salmon Kit - Meyer Lemon & Herb Panko (2 fillets +…
$9.99
scraping  Salmon Cold Smoked
$10.99
scraping Salmon Fillet Coho Marine Stewardship Council
$14.99
scraping  Wild-Caught Cod Fillets
$19.99
scraping  Wild-Caught Breaded Fish Fillets
$5.99
scraping Fishp

29
scraping  Nanas Chicken Noodle Soup
$8.99
scraping Diestel Turkey Ranch
$9.99
scraping Diestel Turkey Ranch
$9.99
scraping Principe
$26.99
scraping  Turkey and Wild Rice Soup
$8.99
scraping Principe
$26.99
scraping Fork In The Road
$11.99
scraping  Double Cheddar Broccoli Soup
$8.99
scraping  Tomato Parmesan Soup
$8.99
scraping Fork In The Road
$11.99
scraping  Red Lentil Dal with Spinach Soup
$8.99
scraping Vincenzas
$14.99
scraping Fork In The Road
$13.99
scraping  Chicken Tortilla Soup
$8.99
scraping 365 Everyday Value
$7.99
scraping Diestel Turkey Ranch
$9.99
scraping Fra Mani
$16.99
scraping  New England Clam Chowder
$7.99
scraping  Pizza Cheese 11 Inch Fresh Pack
$6.99
scraping 365 Everyday Value
$7.99
scraping Whole Foods Market
$10.99
scraping  Falafel Snack Pack
$4.99
scraping Vincenzas
$16.99
scraping Whole Foods Market
$10.99
scraping  Soup Garden Minestrone Fresh Pack
$8.99
scraping  Falafel Snackpack Chipotle Fresh Pack 
$4.99
scraping  Soup Roasted Tomato Basil Fresh P

...

30
scraping  750 mL
$31.99
scraping  Organic Classic Kombucha Original
$3.49
scraping Sutter Home Chardonnay
$5.99
scraping  Kombucha Classic Pina Paradise Organic
$3.49
scraping  750 ml
$29.99
scraping  6 pk
$6.99
scraping  Petite Sirah North Coast
$9.99
scraping  750 ml
$21.99
scraping Stella Rosa Moscato d'Asti
$11.99
scraping Stiegl
$9.99
scraping  750 ml
$54.99
scraping Jadot Pinot Noir
$20.98
scraping Founders All Day IPA
$1.99
scraping  25.36 oz bottle
$9.99
scraping  Malbec
$11.99
scraping  805
$9.49
scraping  Lager
$10.99
scraping  Pinot Grigio
$8.99
scraping Edna Valley Vineyard Chardonnay
$11.99
scraping  12 pk
$14.99
scraping Alamos Cabernet Sauvignon
$10.99
scraping  6 pk
$11.99
scraping Mer Soleil Reserve Chardonnay
$28.99
scraping  Sour Gose Fruitlands Can
$10.99
scraping  Montepulciano D'Abruzzo
$12.99
scraping  Cabernet Sauvignon Organic
$12.99
scraping  Cabernet Sauvignon Non Alcoholic
$8.99
scraping  750 mL
$19.99
scraping  6 pk
$6.99
scraping Merf Cabernet Sauvignon

30
scraping Miyoko's Creamery Classic Double Cream Chive Artisan Vegan Cheese
$8.99
scraping  Gouda Shreds
$4.49
scraping Organic Valley
$6.79
scraping  Cheese Mozzarella Shreds Non Dairy
$4.99
scraping Dutch Cheese Makers
$20.98
scraping Norseland
$6.99
scraping  Fresh Mozzarella Medallions
$4.39
scraping  Pepper Jack Cheese Shredded
$4.49
scraping Organic Valley
$6.79
scraping  Feta Barrel Organic
$5.99
scraping  Cheese Cashew Soft French Style Herb Garlic
$7.49
scraping  Alternative Cheese Provolone Slices
$4.99
scraping 365 Everyday Value
$6.99
scraping El Trigal
$24.99
scraping Emmi Roth
$15.99
scraping  Monterey Jack Shred
$4.49
scraping  Mozzarella Bufala Ovoline
$9.99
scraping  Vegan Cheese Mozzarella Shreds
$4.99
scraping  Organic Shredded Pepper Jack Cheese
$6.99
scraping Kaasaggio
$18.99
scraping Organic Valley
$7.39
scraping Organic Valley
$8.49
scraping  Mozzarella Burrata
$6.99
scraping  Petit Suisse With Fruit
$4.99
scraping Mt Vikos Barrel-Aged Feta
$7.99
scraping  Redu

30
scraping  Chips Krinkle Cut Classic Barbeque
$4.99
scraping  Fire Roasted Red Medium
$4.99
scraping  Organic Popcorn
$2.49
scraping  Almonds - Sliced
$4.99
scraping  Caramel
$1.99
scraping  Backyard BBQ
$2.99
scraping  Chocolate Cherry Carnival Trail Mix
$6.99
scraping  Nacho Bean Chips
$3.68
scraping  Bar Dark Chocolate Chip Peanut Butter Organic
$2.79
scraping Think High Protein Bar Naturally Flavored Cookies and Creme
$1.99
scraping  Salsa Green Medium
$5.99
scraping Dang Toasted Coconut Chips
$4.39
scraping  Oats and Honey with Toasted Coconut
$3.99
scraping 365 Everyday Value Organic Mini Peanut Butter Sandwich Crackers
$2.99
scraping  Mint Chocolate Chip Snack Bar
$2.49
scraping  Blue Chips
$3.99
scraping ANGIE'S BOOMCHICKAPOP Sea Salt Popcorn
$3.69
scraping Lesser Evil Buddha Bowl Himalayan Pink Organic Popcorn
$3.99
scraping  Double Feature Trail Mix
$6.99
scraping Kettle Brand Himalayan Salt Potato Chips Cooked In Avocado Oil
$2.99
scraping  Organic Chia & Flax Ground Seed 

30
scraping  Pistachios Roasted Shelled
$14.99
scraping  Bar Protein Almond Honey Gluten Free
$2.69
scraping  Organic Popcorn
$2.49
scraping 365 Everyday Value Hodgepodge Harvest Trail Mix
$6.99
scraping  Peanut Free
$5.29
scraping  Dark Chocolate
$3.68
scraping  Organic Cape Cod Trail Mix
$9.99
scraping  Organic Fruit Strip
$0.59
scraping  Seedbar Superfood Currant Cacao Hemp Seed
$2.99
scraping  Low-Carb
$3.99
scraping  Molka Fire Roasted Salsa
$3.79
scraping Garden of Eatin' Yellow Corn Tortilla Chips
$5.49
scraping  Butter Shortbread
$3.99
scraping  Berry Bounty Trail Mix
$6.99
scraping  Organic Kettle Corn
$2.49
scraping  Bar Stronger Peanut Cluster
$2.49
scraping  Crisps Parmesan
$7.99
scraping  Bar Fruit Apple Mango
$1.69
scraping  Caramel Sea Salt Chocolate Chip
$4.49
scraping 365 Everyday Value Large Marshmallows
$2.49
scraping  Peanut Butter Cookie
$1.39
scraping Natierra Nature's All Foods Organic Freeze-Dried Bananas
$5.99
scraping  Seedbar Superfood Mulberry Cacao Spirulin

30
scraping  Muffins Wild Blueberry
$6.99
scraping  Organic Ezekiel 4:9 Hamburger Buns
$4.99
scraping  Organic Rice Brown Bread
$5.99
scraping  Bagelettes Honey Wheat
$3.99
scraping  Buns Hamburger White Organic
$5.49
scraping  Bread French
$2.99
scraping  Pancake Waffle Mix Banana Paleo
$5.99
scraping 365 Everyday Value Chocolate Chip Cookie Mix
$2.99
scraping OZERY BAKERY Onebun 100 Calorie Wholewheat Wholegrain Pita Bread
$2.99
scraping  Blondie Cinnamon Roll
$2.29
scraping Dave's Killer Bread Rockin' Grains English Muffins
$6.99
scraping  Muffin Chocolate Chip
$4.99
scraping Rudi's Gluten-Free Multigrain Sandwich Bread
$6.49
scraping Rudi's Organic Country Morning White Bread
$5.79
scraping  Bread Boule Golden Age
$4.99
scraping  Organic Bread Rustic Italian
$1.99
scraping  Bread Pita Apple Cinnmon Snack Round
$3.99
scraping  Bread Ciabatta Organic
$3.99
scraping  Canola Flour Tortilla 10 Count
$2.69
scraping Arrowhead Mills Organic Sprouted Grain Pancake and Waffle Mix
$6.99
scrap

In [584]:
len(product_list)

3258

In [585]:
WholeFoods = pd.DataFrame(product_list)

In [586]:
WholeFoods.to_csv('datasets/wholefoods2.csv', index=False)